<a href="https://colab.research.google.com/github/rudycav/RedditScraper/blob/main/Project/r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
!pip install asyncpraw
!pip install praw
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
import asyncpraw
import praw
from praw.models import MoreComments
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [77]:
reddit = praw.Reddit(
    client_id="BE3T0CMpNrtqLT97yL6XiA",
    client_secret = "gJnvUXTK0gCnj3UMGSUtw0JW35tN1A",
    password = "Hannah00!!@",
    user_agent = "spidey by u/rudyboycav",
    username = "rudyboycav",
)

In [78]:
def reddit_scraper(reddit, subreddit_page, limit):
  df = pd.DataFrame()
  subreddit = reddit.subreddit(subreddit_page)
  hot_topic = subreddit.hot(limit=limit)

  for x in hot_topic: 
    c = x.comments.list()
    for y in c:
      if isinstance(y, MoreComments):
        continue
      df = df.append({'subreddit': subreddit_page,
                      'id': x.id,
                      'title': x.title,
                      'author': x.author,
                      'comments': y.body},
                      ignore_index=True) 

  return df.comments

In [79]:
wall = reddit_scraper(reddit, 'wallstreetbets', 7)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [80]:
print(len(wall))

2407


In [81]:
def tokenize(df):
  for words in wall:
    tokenize = word_tokenize(words)
  return tokenize

tokenized_words = tokenize(wall)

In [82]:
stop_words = set(stopwords.words("english"))

In [83]:
def stopwords(df):
  #stop_words = set(stopwords.words("english"))
  filtered_words = [word for word in df if word.casefold() not in stop_words]
  return filtered_words

filtered_stopwords = stopwords(tokenized_words)
filtered_stopwords

['time', 'Chuck', 'Norris', 'wrong', 'thought', 'made', 'mistake', '...']

In [84]:
def stemming(df):
  stemmed_list = []
  snow_stemmer = SnowballStemmer(language='english')
  
  for words in filtered_stopwords:
    stemmer = snow_stemmer.stem(words)
    stemmed_list.append(stemmer)
  return stemmed_list

stemmed_words = stemming(filtered_stopwords)
stemmed_words

['time', 'chuck', 'norri', 'wrong', 'thought', 'made', 'mistak', '...']

In [85]:
subreddit = await reddit.subreddit("wallstreetbets")
async for x in subreddit.hot(limit=3):
    print('THREAD: {}'.format(x.title))
    print('USER: {}'.format(x.author))

    print(x.comments)
    



TypeError: ignored